In [132]:
!pip install ultralytics

In [133]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

In [134]:
# load yolov8 model after transfer learning
#model_name = 'yolov8n.pt'
model_name = '/content/best_100_epochs_orig.pt'
model = YOLO(model_name)

In [135]:
# map class IDs to class names
vehicles = {0: "auto", 1: "bike", 2: "bus", 3: "car", 4: "truck"}

In [136]:
# load test video
video_path = './dashcam_india_clip1.mp4'
cap = cv2.VideoCapture(video_path)

In [137]:
# read first frame
ret, frame = cap.read()

In [138]:
# set output video format with encodings
format_fourcc = cv2.VideoWriter_fourcc(*'MP4V')

# get frame rate of input video
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
print(frame_rate)

# get frame size of input video
frame_size = (frame.shape[1], frame.shape[0])
print(frame_size)

30
(852, 480)


In [139]:
video_result_path = './dashcam_india_clip1_result.mp4'
cap_result = cv2.VideoWriter(video_result_path, format_fourcc, frame_rate, frame_size)

In [140]:
# compute time interval between frames
time_interval = 1 / frame_rate

In [141]:
# define a set of colours
import random
colours = [(random.randint(0,255), random.randint(0,255), random.randint(0,255)) for j in range(10)]

In [142]:
# store previous bounding box details for each track_id
previous_boxes = {}

In [143]:
ret = True
# read frames
while ret:
  # detect and track objects
  results = model.track(frame, persist=True)

  # plot results
  result = results[0]
  for r in result.boxes.data.tolist():
    # get coordinates, track_id, class_id
    print("Object details: ", r)

    x1, y1, x2, y2, track_id, score, class_id = r
    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)
    track_id = int(track_id)
    class_id = int(class_id)
    class_name = vehicles[class_id]
    colour = colours[track_id % len(colours)]
    line_width = 3
    font_width = 2

    # calculate bounding box center
    bbox_left = min(x1, x2)
    bbox_top = min(y1, y2)
    bbox_width = abs(x2 - x1)
    bbox_height = abs(y2 - y1)

    x_c = bbox_left + (bbox_width / 2)
    y_c = bbox_top + (bbox_height / 2)
    bbox_area = bbox_width * bbox_height

    # get or initialize previous box information for the track_id
    prev_box = previous_boxes.get(track_id, {"center": (x_c, y_c), "area": bbox_area})

    # estimate speed using size change
    area_change = bbox_area - prev_box["area"]
    speed = area_change / time_interval

    # log object and speed information
    print("Track ID: ", track_id)
    print("Class: ", class_name)
    print("Speed: ", speed, "pixel sq. / second")

    # display speed in terms of motion units (MU) per second [1 MU = 10000 pxs/s]
    speed_display = speed/10000

    # Update previous box information
    previous_boxes[track_id] = {"center": (x_c, y_c), "area": bbox_area}

    # plot bounding box with speed information
    cv2.rectangle(frame, (x1, y1), (x2, y2), colour, line_width)
    cv2.putText(frame, f"{class_name} {speed_display:.2f} MU", (x1+10, y1+25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, colour, font_width)

  #add result frame to result video
  cap_result.write(frame)

  #next frame
  ret, frame = cap.read()

Streaming output truncated to the last 5000 lines.
Object details:  [623.3203125, 181.71893310546875, 851.3390502929688, 477.9252014160156, 2.0, 0.8654900193214417, 3.0]
Track ID:  2
Class:  car
Speed:  -6840.0 pixel sq. / second
Object details:  [266.192138671875, 155.36898803710938, 480.07745361328125, 469.2880859375, 3.0, 0.9059224128723145, 0.0]
Track ID:  3
Class:  auto
Speed:  -3390.0 pixel sq. / second

0: 384x640 1 auto, 1 car, 217.1ms
Speed: 4.7ms preprocess, 217.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Object details:  [627.5301513671875, 182.10313415527344, 852.0, 477.50921630859375, 2.0, 0.8585502505302429, 3.0]
Track ID:  2
Class:  car
Speed:  -33390.0 pixel sq. / second
Object details:  [263.17510986328125, 154.52920532226562, 480.6719970703125, 473.7062072753906, 3.0, 0.8829902410507202, 0.0]
Track ID:  3
Class:  auto
Speed:  60810.0 pixel sq. / second

0: 384x640 1 auto, 1 car, 197.2ms
Speed: 3.3ms preprocess, 197.2ms inference, 1.1ms postpro

In [144]:
cap.release()
cap_result.release()